In [1]:
import ifcopenshell
import ifcopenshell.api
import ifcopenshell.api.owner.settings
import numpy as np

In [2]:
nummber = np.arange(10)

In [3]:
nummber

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [4]:
class ifc:
    def __init__(self):
        pass
        
    def create_ifc_file(self, version):
        self.file = ifcopenshell.api.run("project.create_file", version=version)
        person = ifcopenshell.api.run("owner.add_person", self.file)
        person[0] = person.GivenName = None
        person.FamilyName = "user"
        org = ifcopenshell.api.run("owner.add_organisation", self.file)
        org[0] = None
        org.Name = "template"
        user = ifcopenshell.api.run("owner.add_person_and_organisation", self.file, person=person, organisation=org)
        application = ifcopenshell.api.run("owner.add_application", self.file)
        ifcopenshell.api.owner.settings.get_user = lambda ifc: user
        ifcopenshell.api.owner.settings.get_application = lambda ifc: application

        project = ifcopenshell.api.run("root.create_entity", self.file, ifc_class="IfcProject", name="My Project")
        lengthunit = ifcopenshell.api.run("unit.add_si_unit", self.file, unit_type="LENGTHUNIT", name="METRE")
        ifcopenshell.api.run("unit.assign_unit", self.file, units=[lengthunit])
        model = ifcopenshell.api.run("context.add_context", self.file, context_type="Model")
        self.context = ifcopenshell.api.run(
            "context.add_context",
            self.file,
            context_type="Model",
            context_identifier="Body",
            target_view="MODEL_VIEW",
            parent=model,
        )
        site = ifcopenshell.api.run("root.create_entity", self.file, ifc_class="IfcSite", name="My Site")
        building = ifcopenshell.api.run("root.create_entity", self.file, ifc_class="IfcBuilding", name="My Building")
        self.storey = ifcopenshell.api.run(
            "root.create_entity", self.file, ifc_class="IfcBuildingStorey", name="My Storey"
        )
        ifcopenshell.api.run("aggregate.assign_object", self.file, product=site, relating_object=project)
        ifcopenshell.api.run("aggregate.assign_object", self.file, product=building, relating_object=site)
        ifcopenshell.api.run("aggregate.assign_object", self.file, product=self.storey, relating_object=building)

        self.origin = self.file.createIfcAxis2Placement3D(
            self.file.createIfcCartesianPoint((0.0, 0.0, 0.0)),
            self.file.createIfcDirection((0.0, 0.0, 1.0)),
            self.file.createIfcDirection((1.0, 0.0, 0.0)),
        )
        self.placement = self.file.createIfcLocalPlacement(None, self.origin)
        self.history = ifcopenshell.api.run("owner.create_owner_history", self.file)

In [5]:
ifc_file = ifc()
ifc_file.create_ifc_file(version="IFC4")

In [6]:
file = ifc_file.file
storey = ifc_file.storey

In [7]:
for i in nummber:
    guid = ifcopenshell.guid.new()
    name_of_object = f"Object {i}"
    random_nummer = 213
    product = file.create_entity(
                "IfcBuildingElementProxy",
                **{
                    "GlobalId": guid,
                    "Name": name_of_object,
                    "ObjectPlacement": None ,
                    
                }
            )
    ifcopenshell.api.run("spatial.assign_container", file, product=product, relating_structure=storey)
    pse = ifcopenshell.api.run("pset.add_pset", file , product=product, name="My Property")
    ifcopenshell.api.run("pset.edit_pset", file , pset=pse, properties={"Created": f"by script{random_nummer}" , "Object Name": name_of_object })
    

In [8]:
file.write("test_1.ifc")

In [9]:
import ifcpatch

In [10]:
ifcpatch.execute({
        "input": "test_1.ifc",
        "output": "test_2.ifc",
        "recipe": "RecycleNonRootedElements",
        "log": [],
        "arguments": [],})

# Loading IFC file .....
# Loading patch recipe ...
# Patching ...
# Writing patched file ...
# All tasks are complete :-)
